In [108]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

In [109]:
dictionary = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_1000)
parameters = cv2.aruco.DetectorParameters()
parameters.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
ArucoDetector = cv2.aruco.ArucoDetector(dictionary, parameters)

In [110]:
# #aruco test
# im = cv2.imread('frame10.jpg')
# grayScale = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
# (corners1, ids1, rejectedImgPoints) = ArucoDetector.detectMarkers(im)
# print(len(rejectedImgPoints))
# print(corners1, ids1, rejectedImgPoints)

In [112]:
camPicture = cv2.imread("frame14.jpg")
row, col, ch = camPicture.shape
print(camPicture.shape)
refImage = cv2.imread("slide3.PNG")
#resize to 1920x1080
print(refImage.shape)
pts1 = np.float32([[0,0],[1280,0],[0,720],[1280,720]])
pts2 = np.float32([[600,126],[1298,173],[591,566],[1341,544]])
matrix = cv2.getPerspectiveTransform(pts1,pts2)
dst = cv2.warpPerspective(refImage,matrix,(col,row))

def subtractImage(channelScales, base, dist):
    h,s,v = cv2.split(cv2.cvtColor(dist, cv2.COLOR_BGR2HSV))
    h = cv2.multiply(h, channelScales[0])
    s = cv2.multiply(s, channelScales[1])
    v = cv2.multiply(v, channelScales[2])
    dist = cv2.cvtColor(cv2.merge((h,s,v)), cv2.COLOR_HSV2BGR)
    sub = cv2.subtract(base, dist)
    return np.sum(sub)

params = minimize(subtractImage, [1,1,1], args=(camPicture, dst), method='Nelder-Mead', bounds=((0, 5), (0, 2), (0, 1)))
print(params.x)


    

(1080, 1920, 3)
(720, 1280, 3)
[1.64444444 0.         1.        ]
